<a href="https://colab.research.google.com/github/tiennvcs/EE6363_AdvancedML/blob/main/HW1/P3/Task_2_LS_Single_Shot_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Consider least square problem $\min_{x \in \mathbf{R}^3}||y - Ax||_2^2$, where $A = \begin{bmatrix}
  3 & 1 & 2 \\
  1 & 3 & 1 \\
  2 & 1 & 3 \\
\end{bmatrix}
$, and $y = \begin{bmatrix}
  2 \\
  3 \\
  4 \\
\end{bmatrix}$

## Solve by inversion method

To solve above problem by stationarity condition, we need calculate the gradient of $f(x) = ||y - Ax||_2^2$, and set this gradient equal to 0, and then find the solution $x$.

$$
∇_xf(x) = 2(A^TAx - A^Ty)
$$

Set $∇_xf(x) = 0$, we have:
$$
    2(A^TAx - A^Ty) = 0 ⇔   (A^TA)x  = A^Ty ⇔ x  = (A^TA)^{-1}A^Ty
$$
in case of $A^TA$ is inversible.

In [1]:
import numpy as np
from numpy.linalg import inv, svd, norm
from typing import Union

In [2]:
def solve_leastsquare_inversion(A: np.ndarray, y: np.ndarray) -> np.ndarray:
    t = inv((A.T).dot(A))
    x = (t.dot(A.T)).dot(y)
    return x

In [3]:
# Test on A, y
A = np.array([
    [3, 1, 2],
    [1, 3, 1],
    [2, 1, 3],
])

y = np.array([2, 3, 4]).T

In [4]:
xi = solve_leastsquare_inversion(A, y)

In [5]:
print("The solution founded x = {}".format(xi))

The solution founded x = [-0.53846154  0.69230769  1.46153846]


## Solve by stabilized SVD

As abovementioned, we need to set gradient equal to 0. It comes up with the system of linear equations:
$$
    (A^TA)x  = A^Ty
$$
To solve it using SVD, we need decompose $(A^TA) = U S V^T$, then we have general solution using SVD: $$x = VS^{-1}U^TA^Ty$$

In [6]:
def standardSVD(A: np.ndarray) -> Union[np.ndarray, np.ndarray, np.ndarray]:
    U, S, Vt = svd(A)
    a = np.zeros(A.shape)
    np.fill_diagonal(a, S)
    S = a
    return U, S, Vt

In [7]:
def stablizedSVD(A: np.ndarray, epsilon=0.5) -> Union[np.ndarray, np.ndarray, np.ndarray]:
    # Call SVD from scipy
    U, S, Vt = standardSVD(A)
    # Find maximum k that satisfy S_{k, k}/S_{0, 0} >= epsilon
    k = 0
    while k < S.shape[0]:
        if S[k, k]/S[0, 0] < epsilon:
            break
        k += 1
    k = k -1
    # Gather columns of U to form new Us
    Us = U[:, 0:k+1]
    # Gather columns of S to form new Ss
    Ss = S[:k+1, :k+1]
    # Gather rows of Vt to form new Vts
    Vts = Vt[:k+1, :]
    return Us, Ss, Vts

In [8]:
def solve_leastsquare_stabilizedSVD(A: np.ndarray, y: np.ndarray, epsilon=0.5) -> np.ndarray:
    # Calculate matrix W = A^T.A
    W = (A.T).dot(A)
    # Call stablizedSVD to get U, S, V
    Us, Ss, Vts = stablizedSVD(W, epsilon=epsilon)
    print("Singular values after reduced: \n{}".format(Ss))
    # Compute the general solution x = VS^-1U^TA^Ty
    x = (((Vts.T).dot(inv(Ss)).dot(Us.T)).dot(A.T)).dot(y)
    return x

In [9]:
xs = solve_leastsquare_stabilizedSVD(A, y, epsilon=0)

Singular values after reduced: 
[[32.85640646  0.          0.        ]
 [ 0.          5.14359354  0.        ]
 [ 0.          0.          1.        ]]


In [10]:
xn = solve_leastsquare_stabilizedSVD(A, y, epsilon=0.1)

Singular values after reduced: 
[[32.85640646  0.        ]
 [ 0.          5.14359354]]


$\rightarrow$ When choose $\epsilon = 0$, stabilized SVD get solution $x^* = [-0.53846154,  0.69230769,  1.46153846]^T$. It is also the solution found by inversion method previously. There is, however, difference when setting $\epsilon=0.1$, it eliminates the third singular values and absolutely affect on $U_s, Vt_s$. Therefore, the solution $x^*$ founded by applying stabilized SVD, in this case, is approximate solution.

In [11]:
# Define f(x) function
def f(A: np.ndarray, x: np.array, y: np.array):
    return norm(y-A.dot(x))**2

In [12]:
zi = f(A, xi, y)
zs = f(A, xs, y)
zn = f(A, xn, y)

In [13]:
# Show the value of the metric at the optimal points
print("Optimmal value x = {} -> f(x) = {} founded by inversion method".format(xi, zi))
print("Optimmal value x = {} -> f(x) = {} founded by stablized SVD method with epsilon = 0".format(xs, zs))
print("Optimmal value x = {} -> f(x) = {} founded by stablized SVD method with epsilon = 0.1".format(xn, zn))

Optimmal value x = [-0.53846154  0.69230769  1.46153846] -> f(x) = 4.141519752410312e-30 founded by inversion method
Optimmal value x = [-0.53846154  0.69230769  1.46153846] -> f(x) = 3.3526588471893e-30 founded by stablized SVD method with epsilon = 0
Optimmal value x = [0.46153846 0.69230769 0.46153846] -> f(x) = 2.0000000000000004 founded by stablized SVD method with epsilon = 0.1


It very makes sense when we look at the solution $x^* = [-0.53846154,  0.69230769,  1.46153846]^T$ founded by inversion and stabilized SVD method and corresponding value $f(x) ≈ 0$. Actually, it should be 0, but we have concerns with accuarcy calculated by Python . When we decompose $A$ into product of $U, S, V,$(in task 1), we see that the $W$ have 3 positive singular values, so $\text{rank}(A) = 3$. It means, $\text{span}\{A_{:, i}\} , \ i = 1, 2, 3$ form 3D world, or $\mathbf{R}^3$. Obviously, $y \in \mathbf{R}^3$, and $Ax$ is the transformation operation turning $x \in \mathbf{R}^3$ to another vector $z \in \mathbf{R}^3$. The function $f(x) = ||y - Ax||^2_2 = ||y - z||^2_2$ is the distance between $z$ and $y$, so $f(x)$ hit the mimimum value iif $z = y$, or the distance equals $0$.